# Program Overview
This script reads through the OCR output of the London Times Digital Archive by Gale Cengage Learning and returns a heat map showing the relative frequency with which a give word appears on the front page. Originally produced to be used with a Big Data Analysis course at the Digital Humanities Summer Institute in 2016. Orignial idea by Pawel Pomorski.  Initial coding by John Simpson.

# General Methodology
The program uses a rectangular numpy array to represent the front page of a newspaper.  When given an word to search for the program searches through the page scans, finding each word and the area on the page on which the word was found.  All cells within the front page array that overlap with that location are incremented.  When all instances have been found and the all increments made the array is then passed to matplotlib to generate a simple heatmap.

## Initialize Numpy Array
We will use a package called Numpy (Numerical Python) to create what will amount to a grid representing locations on a single page of the London Times.  Numpy has a lot of very nice features for doing vector mathematics, features which we do not really need for this program.  We're using Numpy simply because it provides really convenient control over data structures like the grid we need for this program.

The size that we are casting the array to initially is a guess made by looking at the word results returned in the TDA OCR xml files (see the maximum values inside the <wd pos="W,X,Y,Z">WORD</wd> tags).

We also recast the array from type 'float64' (big decimal numbers) to 'int8' (small integer numbers) since we are only doing simple counts and do not need the extra memory used to track large decimals.  If you would like to see this happen then just uncomment the `#frontPage.dtype` lines when running the code block.


In [1]:
import numpy as np
frontPage = np.zeros((7000,5250)) 
#frontPage.dtype
frontPage = frontPage.astype('uint32')
#frontPage.dtype

## Collect a Search Term
We simply ask the user for word to search through the text of the London times for.  Any input provided is put into lowercase because we will use lower case as the standard for comparison of string values.

Note that as currently written the code will accept strings with spaces from users which is a potential problem because the XML code only has single words.  It is left as a relatively simple exercise to build in an error trapping routine that will inform the user that only *single* words may me entered.  It is left as a rather involved exercise to modify the program so that it can successfully handle the input of phrases.

In [2]:
searchTerm = input("Please enter a single word that you would like to search the London Times for:")
searchTerm = searchTerm.lower()

Please enter a single word that you would like to search the London Times for:war


## Find the Search Term in the Data
Here we open a folder within the directory that this notebook/code is held called `Data/OCRText` and for every file within that directory we look inside to see if it is well formed XML.  If it is then we look inside every `pg` tag with attribute `pgref="1"` and scan through all its child tags to find the search term.  If it is found then the index values for the term are used up increment the appropriate values within our array, `frontPage`.

This StackOverflow discussion was useful in walking the data directory with this code: [http://stackoverflow.com/questions/3964681/find-all-files-in-directory-with-extension-txt-in-python](http://stackoverflow.com/questions/3964681/find-all-files-in-directory-with-extension-txt-in-python)

This online tool was useful in preparing the XPATH to target and return the right portion of the xml tree: [http://www.freeformatter.com/xpath-tester.html](http://www.freeformatter.com/xpath-tester.html)

`lxml` uses XPATH 1.0 which means that we don't have access to the the function `lowercase()` (it is not in the library `xpath2_funcitons`).  As a consequence it was necessary to use the function `translate()` in order to convert all regular characters to their lower case equivalent.  Special characters will be overlooked by this and so some words may be ignored.

For help slicing arrays see: [http://structure.usc.edu/numarray/node26.html](http://structure.usc.edu/numarray/node26.html)

In [3]:
import os
ocrData = os.path.dirname("Data/OCRText39-45/")

from lxml import etree

def searchXML(searchTerm, file):
    #print(file)
    try:
        tree = etree.parse(file)
        MyXpath = '//p[preceding-sibling::pg[@pgref=1]]/wd[translate(text(), "ABCDEFGHIJKLMNOPQRSTUVWXYZ", "abcdefghijklmnopqrstuvwxyz")="' + searchTerm + '"]/@pos'

        find_text = etree.XPath(MyXpath)
        # print(find_text)
        foundPOS = find_text(tree)
        # print(foundPOS)
        if foundPOS != []:
            for pos in foundPOS:
                posList = [int(item) for item in str(pos).split(',')]
                #print(pos,posList)
                frontPage[posList[1]-1:posList[3],posList[0]-1:posList[2]] += 1
                #print(frontPage[posList[0]-2:posList[2]+1,posList[1]-2:posList[3]+1])
    except:
        print("Likely XMLSyntax Error in", file, ", skipping") 

for root, dirs, files in os.walk(ocrData):
    for file in files:
        if file.endswith(".xml") and file.startswith("0FFO"):
            #print(os.path.join(root, file))
            searchXML(searchTerm,os.path.join(root, file))
            

Likely XMLSyntax Error in Data/OCRText39-45/1939/19390130/0FFO-1939-JAN30.xml , skipping
Likely XMLSyntax Error in Data/OCRText39-45/1939/19390204/0FFO-1939-FEB04.xml , skipping
Likely XMLSyntax Error in Data/OCRText39-45/1939/19390809/0FFO-1939-AUG09.xml , skipping
Likely XMLSyntax Error in Data/OCRText39-45/1940/19400801/0FFO-1940-AUG01.xml , skipping
Likely XMLSyntax Error in Data/OCRText39-45/1940/19401011/0FFO-1940-OCT11.xml , skipping
Likely XMLSyntax Error in Data/OCRText39-45/1940/19401025/0FFO-1940-OCT25.xml , skipping
Likely XMLSyntax Error in Data/OCRText39-45/1940/19401115/0FFO-1940-NOV15.xml , skipping
Likely XMLSyntax Error in Data/OCRText39-45/1940/19401231/0FFO-1940-DEC31.xml , skipping
Likely XMLSyntax Error in Data/OCRText39-45/1941/19410131/0FFO-1941-JAN31.xml , skipping
Likely XMLSyntax Error in Data/OCRText39-45/1942/19420509/0FFO-1942-MAY09.xml , skipping
Likely XMLSyntax Error in Data/OCRText39-45/1942/19420916/0FFO-1942-SEP16.xml , skipping
Likely XMLSyntax Erro

## Produce the Heat Map
Since we have all the data in an array producing the heat map is easily done using the matplotlib package.  

Note the use of the `% matplotlib inline` line to tell Jupyter Notebooks to produce charts *in the notebook* and not in a separate window (which can sometimes open behind the current window or in another workspace and leave users scracting their head and wondering why it looks like the plot failed).

In [4]:
import matplotlib.pyplot as plt
% matplotlib inline
plt.figure(figsize=(500, 200))
image  = plt.imshow(frontPage)
plt.show()

In [ ]:
a1 = np.ones((3, 3))
a2 = np.ones((12, 12))
#np.add.at(a2, [[1,2],[4,5]], 4)
a2[5:8,1:4] += 1
a2